In [2]:
!pip install python-telegram-bot

  Using cached https://files.pythonhosted.org/packages/26/aa/82dfedea526128a55429527ca5e43c023846e00cc3df30c8dbfebe94bbb2/python_telegram_bot-12.2.0-py2.py3-none-any.whl


In [3]:
import logging
import pandas as pd
import re

from telegram import ReplyKeyboardMarkup, InlineKeyboardButton, InlineKeyboardMarkup
from telegram.ext import (Updater, CommandHandler, MessageHandler, Filters,
                          ConversationHandler, CallbackQueryHandler)


# Enable logging
logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
                    level=logging.INFO)

logger = logging.getLogger(__name__)

(START, FIND_SPEC, ABOUT, FIND_SPEC_NOW, FIND_SPEC_LATER, SPEC_LIST, CHOOSE_AGE, PROBLEM_CATEGORY, 
 QUESTION_CATEGORY, CHOOSING_ABOUT) = range(10)

class Psy_Chat:
    def __init__(self, specialist_data_route):
        specialist_data = pd.read_csv(specialist_data_route)
        
    def start(self, update, context):
        user = update.effective_user.first_name
        logger.info("User %s goes to find_spec section.", user)
        # Build InlineKeyboard where each button has a displayed text
        # and a string as callback_data
        # The keyboard is a list of button rows, where each row is in turn
        # a list (hence `[[...]]`).
        keyboard = [
            [InlineKeyboardButton('Найти специалиста', callback_data=str(FIND_SPEC)),
             InlineKeyboardButton('О сервисе', callback_data=str(ABOUT))]
        ]
        reply_markup = InlineKeyboardMarkup(keyboard)
        # Send message with text and appended InlineKeyboard
        update.effective_chat.send_message(
            text = """Привет!\n\nЭтот бот поможет тебе справиться с депрессией или другими психологическими проблемами, которые тебя беспокоят!\n\nДавай начнем: выбери "Найти специалиста" или узнай о нас больше выбрав "О сервисе", так же ты можешь выбрать быструю помощь и мы найдем тебе ближайшего освободившегося специалиста""",
            reply_markup=reply_markup
    )
        return START

    
    def stop(self, update, context):
        """Закончить общение с ботом по команде /stop"""
        update.message.reply_text("""Пока!""")

        return ConversationHandler.END
    
    def done(self, update, context):
        update.message.reply_text("""Пока!""")
        user_data = context.user_data
        if 'choice' in user_data:
            del user_data['choice']

        print(user_data)

        user_data.clear()
        return ConversationHandler.END
    
    def find_spec(self, update, context):
        user = update.effective_user.first_name
        logger.info("User %s goes to find_spec section.", user)
        #Update message and keyboard
        query = update.callback_query
        bot = context.bot
        keyboard = [
            [InlineKeyboardButton('Быстрая помощь', callback_data=str(FIND_SPEC_NOW))],
            [InlineKeyboardButton('Подобрать специалиста', callback_data=str(FIND_SPEC_LATER))],
            [InlineKeyboardButton('Список специалистов', callback_data=str(SPEC_LIST))],
        #         [InlineKeyboardButton('Назад', callback_data=str(START))]
        ]
        reply_markup = InlineKeyboardMarkup(keyboard)
        bot.edit_message_text(
            chat_id=query.message.chat_id,
            message_id=query.message.message_id,
            text="Мы можем помочь подобрать тебе специалиста сразу или в зависимости от характера твоей проблемы, либо ты можешь выбрать его самостоятельно из списка специалистов.",
            reply_markup=reply_markup
            )

        return FIND_SPEC
    
    def find_spec_later(self, update, context):
    
        #IN PROGRESS

        # Section for complex search of specs
        user = update.effective_user.first_name
        logger.info("User %s goes to find_spec_later section.", user) 

        context.query = update.callback_query
        context.bot = context.bot

        context.user_data = pd.DataFrame(index = [update.effective_user.id], columns = ['gender', 'age', 'city', 'problem_category', 'statement_category'])
        for data in user_data.columns:
            if data == 'gender':
                def gender(update, context):
                    keyboard = [
                        [InlineKeyboardButton('Мужской', callback_data = "Мужской"),
                         InlineKeyboardButton('Женский', callback_data = "Женский")]
                    ]
                    reply_markup = InlineKeyboardMarkup(keyboard)
                    # Send message with text and appended InlineKeyboard
                    update.effective_chat.send_message(
                        text = "Твой пол?",
                        reply_markup=reply_markup
                    )
                    print(update.callback_query)
                    while update.inline_query == None:
                        print(update.callback_query)
                        user_data.loc[update.effective_user.id, 'gender'] = update.inline_query.query
                        print(user_data.loc[update.effective_user.id, 'gender'])
        pass
    
    def spec_list(self, update, context):
        pass
    
    
    @run_async
    def roulette_command(self, bot, update):
        user_id = update.message.from_user.id
        user = self.store.get_user(user_id)

        self.analytics.track(user_id, update.message.to_dict(), 'roulette_command')

        if user is None:
            user = self.store.create_user(user_id)

        if user['chat_with'] and user['chat_with'] > 0:
            paired_user_id = user['chat_with']
            self.store.disconnect(user_id, paired_user_id)
            bot.sendMessage(user_id, text=_('DISCONNECTED_SEARCH_NEW'), reply_markup=KEYBOARD_MARKUP)
            bot.sendMessage(paired_user_id, text=_('DISCONNECTED'), reply_markup=KEYBOARD_MARKUP)

        paired_user_id = self.store.pair_user(user_id, lambda users: random.choice(users))

        if paired_user_id is None:
            bot.sendMessage(user_id, text=_('SEARCHING'), reply_markup=KEYBOARD_MARKUP)
        else:
            logger.info('Paired ' + str(update.message.from_user) + ' with ' + str(paired_user_id))
            bot.sendMessage(user_id, text=_('ESTABLISHED'), reply_markup=KEYBOARD_MARKUP)
            bot.sendMessage(paired_user_id, text=_('ESTABLISHED'), reply_markup=KEYBOARD_MARKUP)

    @run_async
    def message(self, bot, update):
        user_id = update.message.from_user.id

        user = self.store.get_user(user_id)
        if user:
            paired_user_id = user['chat_with']

            if paired_user_id == STATE_SEARCH:
                self.analytics.track(user_id, update.message.to_dict(), 'error_search')
                bot.sendMessage(user_id, text=_('ERROR_SEARCHING'), reply_markup=KEYBOARD_MARKUP)

            if paired_user_id == STATE_IDLE:
                self.analytics.track(user_id, update.message.to_dict(), 'error_idle')
                bot.sendMessage(user_id, text=_('ERROR_IDLE'), reply_markup=KEYBOARD_MARKUP)

            if paired_user_id and paired_user_id > 0:
                logger.info('Resending ' + str(update.message))
                self.analytics.track(user_id, update.message.to_dict(), 'message')
                try:
                    resend_message(bot, update.message, paired_user_id, format_message)
                except Unauthorized:
                    bot.sendMessage(user_id, text=_('DISCONNECTED'), reply_markup=KEYBOARD_MARKUP)
                    self.store.disconnect(user_id, None)
                    self.store.disconnect(paired_user_id, None)
                    
                    
    def gender(self, update, context):
        r"""Пол"""
        reply_keyboard_loc = [['М', 'Ж']]
        markup_loc = ReplyKeyboardMarkup(reply_keyboard_loc, one_time_keyboard=True)

        update.message.reply_text("""Выберите ваш пол.""", reply_markup=markup_loc)
        return CHOOSE_AGE


    def age(self, update, context):
        r"""Возраст"""
        reply_keyboard_loc = [['< 16', '16-22', '22-40', "> 40"]]
        markup_loc = ReplyKeyboardMarkup(reply_keyboard_loc, one_time_keyboard=True)
        update.message.reply_text("""Выберите ваш возраст.""", reply_markup=markup_loc)

        return PROBLEM_CATEGORY

    def problem_category(self, update, context):
        r"""Выбрать проблему из предложенных типов"""
        reply_keyboard_loc = [['Семья', 'Друзья', 'Отношения', "Работа", "Учеба", "Другое"]]
        markup_loc = ReplyKeyboardMarkup(reply_keyboard_loc, one_time_keyboard=True)
        update.message.reply_text("""К какой категории Вы могли бы отнести свою психологическую проблему?""", reply_markup=markup_loc)

        return QUESTION_CATEGORY

    def statement_category(self, update, context):
        r"""Выбрать утверждение из предложенных типов"""
        reply_keyboard_loc = [['Я запутался/лась', 'Мне не интересно или скучно', 'Я устал', "Меня никто не слушает",
                               "Что делать дальше", "Как сделать правильный выбор", "Другое"]]
        markup_loc = ReplyKeyboardMarkup(reply_keyboard_loc, one_time_keyboard=True)
        update.message.reply_text("""Какие утверждения или вопросы подходят Вам наиболее всего?""", reply_markup=markup_loc)

        return WAITING_TIME

    def find_spec_now(self, update, context):
        r"""Подбор специалиста сразу"""
        query = update.callback_query
        bot = context.bot
        keyboard = [
            [InlineKeyboardButton('Отменить поиск', callback_data=str(START))]
        ]
        reply_markup = InlineKeyboardMarkup(keyboard)
        bot.edit_message_text(
            chat_id=query.message.chat_id,
            message_id=query.message.message_id,
            text="Мы уже ищем специалиста. Ожидайте...",
            reply_markup=reply_markup
        )

        ''' Нужно допилить алгоритм быстрого поиска'''
        def fast_search(self, update, context):
            pass

        return FIND_SPEC_NOW


    def about(self, update, context):
        r"""О нас"""
        reply_keyboard_loc = [['О сервисе', 'Стоимость услуг', 'Отзывы', "О нас пишут", "FAQ" ]]
        markup_loc = ReplyKeyboardMarkup(reply_keyboard_loc, one_time_keyboard=True)
        update.message.reply_text("""PsyHelp - бот психологической поддержки""", reply_markup=markup_loc)

        return CHOOSING_ABOUT

    def error(self, update, context):
        """Log Errors caused by Updates."""
        logger.warning('Update "%s" caused error "%s"', update, context.error)
        
    def run(self):

        updater = Updater("984001690:AAHvJPz7H064B0J9CTJE-243qYMSOVhisPs", use_context=True)

        # Get the dispatcher to register handlers
        dp = updater.dispatcher

        conv_handler = ConversationHandler(
            entry_points=[CommandHandler('start', self.start)],
            states={
                START: [CallbackQueryHandler(self.find_spec, pattern='^' + str(FIND_SPEC) + '$'),
                       CallbackQueryHandler(self.about, pattern='^' + str(ABOUT) + '$')],

                FIND_SPEC: [CallbackQueryHandler(self.find_spec_now, pattern='^' + str(FIND_SPEC_NOW) + '$'),
                           CallbackQueryHandler(self.find_spec_later, pattern='^' + str(FIND_SPEC_LATER) + '$'),
                           CallbackQueryHandler(self.spec_list, pattern='^' + str(SPEC_LIST) + '$')],

    #             FIND_SPEC_NOW: [CallbackQueryHandler(start, pattern='^' + str(START) + '$')],

    #             ABOUT: [CallbackQueryHandler(start_over, pattern='^' + str(ONE) + '$'),
    #                      CallbackQueryHandler(end, pattern='^' + str(TWO) + '$')]
            },
            fallbacks=[CommandHandler('start', start),
                      CallbackQueryHandler(start, pattern='^' + str(START) + '$')
                      ]
        )    

        dp.add_handler(conv_handler)

        # log all errors
        dp.add_error_handler(error)

        # Start the Bot
        updater.start_polling()

        # Run the bot until you press Ctrl-C or the process receives SIGINT,
        # SIGTERM or SIGABRT. This should be used most of the time, since
        # start_polling() is non-blocking and will stop the bot gracefully.
        updater.idle()

In [4]:
a = Psy_Chat()

In [5]:
a.run()

C:\Users\gpbu6399\AppData\Local\Continuum\anaconda3\lib\site-packages\telegram\ext\conversationhandler.py:215: UserWarning: If 'per_message=False', 'CallbackQueryHandler' will not be tracked for every message.
  warnings.warn("If 'per_message=False', 'CallbackQueryHandler' will not be "
2019-12-11 17:57:15,769 - telegram.vendor.ptb_urllib3.urllib3.connectionpool - WARNING - Retrying (Retry(total=2, connect=None, read=None, redirect=None)) after connection broken by 'ConnectTimeoutError(<telegram.vendor.ptb_urllib3.urllib3.connection.VerifiedHTTPSConnection object at 0x0000023CD15BBF98>, 'Connection to api.telegram.org timed out. (connect timeout=5.0)')': /bot984001690:AAHvJPz7H064B0J9CTJE-243qYMSOVhisPs/getMe
2019-12-11 17:57:20,772 - telegram.vendor.ptb_urllib3.urllib3.connectionpool - WARNING - Retrying (Retry(total=1, connect=None, read=None, redirect=None)) after connection broken by 'ConnectTimeoutError(<telegram.vendor.ptb_urllib3.urllib3.connection.VerifiedHTTPSConnection object

NetworkError: urllib3 HTTPError HTTPSConnectionPool(host='api.telegram.org', port=443): Max retries exceeded with url: /bot984001690:AAHvJPz7H064B0J9CTJE-243qYMSOVhisPs/getMe (Caused by ConnectTimeoutError(<telegram.vendor.ptb_urllib3.urllib3.connection.VerifiedHTTPSConnection object at 0x0000023CD15BC3C8>, 'Connection to api.telegram.org timed out. (connect timeout=5.0)'))

In [2]:
import logging
import pandas as pd
import re

from telegram import ReplyKeyboardMarkup, InlineKeyboardButton, InlineKeyboardMarkup
from telegram.ext import (Updater, CommandHandler, MessageHandler, Filters,
                          ConversationHandler, CallbackQueryHandler)


# Enable logging
logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
                    level=logging.INFO)

logger = logging.getLogger(__name__)

(START, FIND_SPEC, ABOUT, FIND_SPEC_NOW, FIND_SPEC_LATER, SPEC_LIST, CHOOSE_AGE, PROBLEM_CATEGORY, 
 QUESTION_CATEGORY, CHOOSING_ABOUT) = range(10)




def start(update, context):
    user = update.effective_user.first_name
    logger.info("User %s goes to find_spec section.", user)
    # Build InlineKeyboard where each button has a displayed text
    # and a string as callback_data
    # The keyboard is a list of button rows, where each row is in turn
    # a list (hence `[[...]]`).
    keyboard = [
        [InlineKeyboardButton('Найти специалиста', callback_data=str(FIND_SPEC)),
         InlineKeyboardButton('О сервисе', callback_data=str(ABOUT))]
    ]
    reply_markup = InlineKeyboardMarkup(keyboard)
    # Send message with text and appended InlineKeyboard
    update.effective_chat.send_message(
        text = """Привет!\n\nЭтот бот поможет тебе справиться с депрессией или другими психологическими проблемами, которые тебя беспокоят!\n\nДавай начнем: выбери "Найти специалиста" или узнай о нас больше выбрав "О сервисе", так же ты можешь выбрать быструю помощь и мы найдем тебе ближайшего освободившегося специалиста""",
        reply_markup=reply_markup
    )
    return START


def stop(update, context):
    """Закончить общение с ботом по команде /stop"""
    update.message.reply_text("""Пока!""")
    
    return ConversationHandler.END

def done(update, context):
    update.message.reply_text("""Пока!""")
    user_data = context.user_data
    if 'choice' in user_data:
        del user_data['choice']

    print(user_data)

    user_data.clear()
    return ConversationHandler.END

def find_spec(update, context):
    user = update.effective_user.first_name
    logger.info("User %s goes to find_spec section.", user)
    #Update message and keyboard
    query = update.callback_query
    bot = context.bot
    keyboard = [
        [InlineKeyboardButton('Быстрая помощь', callback_data=str(FIND_SPEC_NOW))],
        [InlineKeyboardButton('Подобрать специалиста', callback_data=str(FIND_SPEC_LATER))],
        [InlineKeyboardButton('Список специалистов', callback_data=str(SPEC_LIST))],
#         [InlineKeyboardButton('Назад', callback_data=str(START))]
    ]
    reply_markup = InlineKeyboardMarkup(keyboard)
    bot.edit_message_text(
        chat_id=query.message.chat_id,
        message_id=query.message.message_id,
        text="Мы можем помочь подобрать тебе специалиста сразу или в зависимости от характера твоей проблемы, либо ты можешь выбрать его самостоятельно из списка специалистов.",
        reply_markup=reply_markup
        )
    
    return FIND_SPEC

def find_spec_later(update, context):
    
    #IN PROGRESS
    
    # Section for complex search of specs
    user = update.effective_user.first_name
    logger.info("User %s goes to find_spec_later section.", user) 
    
    context.query = update.callback_query
    context.bot = context.bot
    
    context.user_data = pd.DataFrame(index = [update.effective_user.id], columns = ['gender', 'age', 'city', 'problem_category', 'statement_category'])
    for data in user_data.columns:
        if data == 'gender':
            def gender(update, context):
                keyboard = [
                    [InlineKeyboardButton('Мужской', callback_data = "Мужской"),
                     InlineKeyboardButton('Женский', callback_data = "Женский")]
                ]
                reply_markup = InlineKeyboardMarkup(keyboard)
                # Send message with text and appended InlineKeyboard
                update.effective_chat.send_message(
                    text = "Твой пол?",
                    reply_markup=reply_markup
                )
                print(update.callback_query)
                while update.inline_query == None:
                    print(update.callback_query)
                    user_data.loc[update.effective_user.id, 'gender'] = update.inline_query.query
                    print(user_data.loc[update.effective_user.id, 'gender'])
    pass

def spec_list(update, context):
    pass

def gender(update, context):
    r"""Пол"""
    reply_keyboard_loc = [['М', 'Ж']]
    markup_loc = ReplyKeyboardMarkup(reply_keyboard_loc, one_time_keyboard=True)
    
    update.message.reply_text("""Выберите ваш пол.""", reply_markup=markup_loc)
    return CHOOSE_AGE


def age(update, context):
    r"""Возраст"""
    reply_keyboard_loc = [['< 16', '16-22', '22-40', "> 40"]]
    markup_loc = ReplyKeyboardMarkup(reply_keyboard_loc, one_time_keyboard=True)
    update.message.reply_text("""Выберите ваш возраст.""", reply_markup=markup_loc)
    
    return PROBLEM_CATEGORY

def problem_category(update, context):
    r"""Выбрать проблему из предложенных типов"""
    reply_keyboard_loc = [['Семья', 'Друзья', 'Отношения', "Работа", "Учеба", "Другое"]]
    markup_loc = ReplyKeyboardMarkup(reply_keyboard_loc, one_time_keyboard=True)
    update.message.reply_text("""К какой категории Вы могли бы отнести свою психологическую проблему?""", reply_markup=markup_loc)
    
    return QUESTION_CATEGORY

def statement_category(update, context):
    r"""Выбрать утверждение из предложенных типов"""
    reply_keyboard_loc = [['Я запутался/лась', 'Мне не интересно или скучно', 'Я устал', "Меня никто не слушает",
                           "Что делать дальше", "Как сделать правильный выбор", "Другое"]]
    markup_loc = ReplyKeyboardMarkup(reply_keyboard_loc, one_time_keyboard=True)
    update.message.reply_text("""Какие утверждения или вопросы подходят Вам наиболее всего?""", reply_markup=markup_loc)
    
    return WAITING_TIME

def find_spec_now(update, context):
    r"""Подбор специалиста сразу"""
    query = update.callback_query
    bot = context.bot
    keyboard = [
        [InlineKeyboardButton('Отменить поиск', callback_data=str(START))]
    ]
    reply_markup = InlineKeyboardMarkup(keyboard)
    bot.edit_message_text(
        chat_id=query.message.chat_id,
        message_id=query.message.message_id,
        text="Мы уже ищем специалиста. Ожидайте...",
        reply_markup=reply_markup
    )
    
    ''' Нужно допилить алгоритм быстрого поиска'''
    def fast_search(update, context):
        pass
    
    return FIND_SPEC_NOW

    
def about(update, context):
    r"""О нас"""
    reply_keyboard_loc = [['О сервисе', 'Стоимость услуг', 'Отзывы', "О нас пишут", "FAQ" ]]
    markup_loc = ReplyKeyboardMarkup(reply_keyboard_loc, one_time_keyboard=True)
    update.message.reply_text("""PsyHelp - бот психологической поддержки""", reply_markup=markup_loc)
    
    return CHOOSING_ABOUT
    
def error(update, context):
    """Log Errors caused by Updates."""
    logger.warning('Update "%s" caused error "%s"', update, context.error)


def main():
    # Create the Updater and pass it your bot's token.
    # Make sure to set use_context=True to use the new context based callbacks
    # Post version 12 this will no longer be necessary
    updater = Updater("984001690:AAHvJPz7H064B0J9CTJE-243qYMSOVhisPs", use_context=True)

    # Get the dispatcher to register handlers
    dp = updater.dispatcher

    # Add conversation handler with the states CHOOSING, TYPING_CHOICE and TYPING_REPLY
#     conv_handler2 = ConversationHandler(
#         entry_points=[CommandHandler('start', start)],

#         states={
#             CHOOSING: [MessageHandler(Filters.regex('^Найти специалиста$'),
#                                       Find_Spec),
#                        MessageHandler(Filters.regex('^О сервисе$'),
#                                       about),
#                        ],
#             STEP_2_FIND_SPEC: [ MessageHandler(Filters.regex('^Быстрая помощь$'),
#                                       find_spec_now),
#                                MessageHandler(Filters.regex('^Подобрать$'),
#                                       Pol),
#                        MessageHandler(Filters.regex('^Список Специалистов$'),
#                                       Pol)
#             ],

#             CHOOSE_AGE: [MessageHandler(Filters.regex('^.*$'), Age) 
#                             ],
            
#             PROBLEM_CATEGORY: [MessageHandler(Filters.regex('^.*$'), Problem_Category)
#                             ],
            
#             QUESTION_CATEGORY: [MessageHandler(Filters.regex('^.*$'), Question_Category)
#                             ],
#         },

#         fallbacks=[MessageHandler(Filters.regex('^Done$'), done),
#                   CommandHandler('stop', stop)]
#     )
    
    conv_handler = ConversationHandler(
        entry_points=[CommandHandler('start', start)],
        states={
            START: [CallbackQueryHandler(find_spec, pattern='^' + str(FIND_SPEC) + '$'),
                   CallbackQueryHandler(about, pattern='^' + str(ABOUT) + '$')],
            
            FIND_SPEC: [CallbackQueryHandler(find_spec_now, pattern='^' + str(FIND_SPEC_NOW) + '$'),
                       CallbackQueryHandler(find_spec_later, pattern='^' + str(FIND_SPEC_LATER) + '$'),
                       CallbackQueryHandler(spec_list, pattern='^' + str(SPEC_LIST) + '$')],
           
#             FIND_SPEC_NOW: [CallbackQueryHandler(start, pattern='^' + str(START) + '$')],
            
#             ABOUT: [CallbackQueryHandler(start_over, pattern='^' + str(ONE) + '$'),
#                      CallbackQueryHandler(end, pattern='^' + str(TWO) + '$')]
        },
        fallbacks=[CommandHandler('start', start),
                  CallbackQueryHandler(start, pattern='^' + str(START) + '$')
                  ]
    )    

    dp.add_handler(conv_handler)

    # log all errors
    dp.add_error_handler(error)

    # Start the Bot
    updater.start_polling()

    # Run the bot until you press Ctrl-C or the process receives SIGINT,
    # SIGTERM or SIGABRT. This should be used most of the time, since
    # start_polling() is non-blocking and will stop the bot gracefully.
    updater.idle()


if __name__ == '__main__':
    main()

2019-12-11 17:25:53,395 - telegram.vendor.ptb_urllib3.urllib3.connectionpool - WARNING - Retrying (Retry(total=2, connect=None, read=None, redirect=None)) after connection broken by 'ProtocolError('Connection aborted.', ConnectionResetError(10054, 'Удаленный хост принудительно разорвал существующее подключение', None, 10054, None))': /bot984001690:AAHvJPz7H064B0J9CTJE-243qYMSOVhisPs/getMe
2019-12-11 17:25:58,397 - telegram.vendor.ptb_urllib3.urllib3.connectionpool - WARNING - Retrying (Retry(total=1, connect=None, read=None, redirect=None)) after connection broken by 'ConnectTimeoutError(<telegram.vendor.ptb_urllib3.urllib3.connection.VerifiedHTTPSConnection object at 0x0000023CD05E1EF0>, 'Connection to api.telegram.org timed out. (connect timeout=5.0)')': /bot984001690:AAHvJPz7H064B0J9CTJE-243qYMSOVhisPs/getMe
2019-12-11 17:26:03,398 - telegram.vendor.ptb_urllib3.urllib3.connectionpool - WARNING - Retrying (Retry(total=0, connect=None, read=None, redirect=None)) after connection broke

NetworkError: urllib3 HTTPError HTTPSConnectionPool(host='api.telegram.org', port=443): Max retries exceeded with url: /bot984001690:AAHvJPz7H064B0J9CTJE-243qYMSOVhisPs/getMe (Caused by ConnectTimeoutError(<telegram.vendor.ptb_urllib3.urllib3.connection.VerifiedHTTPSConnection object at 0x0000023CD05D50B8>, 'Connection to api.telegram.org timed out. (connect timeout=5.0)'))